# README




*   Data: files under:
    * /content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/Experiment Data (Unformatted)

*   Example:
    * /content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/Experiment Data (Unformatted)/Calvin/Calvin_10_DoubleRoundabout_NewSession_2025.03.07_15.05.22/0.data_2025.03.07_15.05.22.csv

*   General data procesing located here:
    * https://github.com/christopheroka/fydp-roundadoubt/blob/main/AVRIL_data_analysis/Velocity_Acceleration_Analysis.ipynb

*   Variables:
    * VI_CarRealTime.Outputs.chassis_accelerations.longitudinal
    * VI_CarRealTime.Outputs.chassis_velocities.longitudinal
    * Roundabout.EgoPedEuclideanDistance

*   Representing:
    * acceleration
    * velocity
    * distance to ped


*   Steps:
1.   get same length dfs for all of the trials
2.   so you have a, b,c persons as columns and n rows for timestamps


*  Sample Pseudcode:
```
for 1 person across n csvs
  read csv  
  get 3 columsn in df
  add to main df

find n = max distance from ped

do n before ped and n after ped
```












# Imports

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 74.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import FormatStrFormatter

import os
import re

import scipy.stats as stats
import seaborn as sns
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pickle

In [3]:
# py -m venv .venv


# # slashes mist be backwards
# .venv\Scripts\activate.bat

# pip install  -r requirements.txt


# pip install pd
# pip install numpy

# Downloads

# pip3 freeze > Downloads\requirements.txt



In [4]:
# df_calvin = pd.read_csv("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/Experiment Data (Unformatted)/Calvin/Calvin_10_DoubleRoundabout_NewSession_2025.03.07_15.05.22/0.data_2025.03.07_15.05.22.csv", encoding='ISO-8859-1', sep=';')
# df_calvin_slim = slim_data(df_calvin)
# print("calvin size")
# print(len(df_calvin_slim.index))

# Core

## 1. Trial Information

In [5]:
trial_IVs = {
    1: {
        "Driver's Direction": "Left",
        "Alert Combination": "Sign 2 + crosswalk",
        "Pedestrian Crossing": "Straight ahead"
    },
    2: {
        "Driver's Direction": "Straight (left lane)",
        "Alert Combination": "Crosswalk",
        "Pedestrian Crossing": "To driver’s left"
    },
    3: {
        "Driver's Direction": "Straight (right lane)",
        "Alert Combination": "No alert",
        "Pedestrian Crossing": "To driver’s left"
    },
    4: {
        "Driver's Direction": "Straight (right lane)",
        "Alert Combination": "Sign 2 + crosswalk",
        "Pedestrian Crossing": "To driver’s right"
    },
    5: {"Driver's Direction": "Left",
        "Alert Combination": "Sign 3",
        "Pedestrian Crossing": "To driver’s left"
    },
    6: {
        "Driver's Direction": "Straight (left lane)",
        "Alert Combination": "No alert",
        "Pedestrian Crossing": "To driver’s right"
    },
    7: {
        "Driver's Direction": "Left",
        "Alert Combination": "Sign 3 + crosswalk",
        "Pedestrian Crossing": "To driver’s left"
    },
    8: {
        "Driver's Direction": "Left",
        "Alert Combination": "Sign 3 + crosswalk",
        "Pedestrian Crossing": "Straight ahead"
    },
    9: {
        "Driver's Direction": "Straight (right lane)",
        "Alert Combination": "Sign 3",
        "Pedestrian Crossing": "Straight ahead"
    },
    10: {
        "Driver's Direction": "Right",
        "Alert Combination": "Crosswalk",
        "Pedestrian Crossing": "Straight ahead"
    },
    11: {
        "Driver's Direction": "Left",
        "Alert Combination": "Sign 2",
        "Pedestrian Crossing": "To driver’s right"
    },
    12: {
        "Driver's Direction": "Right",
        "Alert Combination": "Sign 3",
        "Pedestrian Crossing": "To driver’s right"
    },
    13: {
        "Driver's Direction": "Left",
        "Alert Combination": "Crosswalk",
        "Pedestrian Crossing": "To driver’s left"
    },
    14: {
        "Driver's Direction": "Right",
        "Alert Combination": "Sign 2",
        "Pedestrian Crossing": "To driver’s left"
    },
    15: {
        "Driver's Direction": "Straight (left lane)",
        "Alert Combination": "Sign 2",
        "Pedestrian Crossing": "Straight ahead"
    },
}

## 2. Functions for Data Cleaning:
Time is currently organized by STEP. Given a frequency of 100Hz, re-organize into millisecond timesteps starting at zero


In [6]:
def slim_data(df):
  sampling_rate = 100
  step_interval_ms = 1000 / sampling_rate

  df['Milliseconds'] = (df.index * step_interval_ms).astype(int)
  df = df[['Milliseconds','VI_CarRealTime.Outputs.chassis_accelerations.longitudinal','VI_CarRealTime.Outputs.chassis_velocities.longitudinal','Roundabout.EgoPedEuclideanDistance' ]]

  # Find minimum non-zero value and its row index
  non_zero_values = df['Roundabout.EgoPedEuclideanDistance'][df['Roundabout.EgoPedEuclideanDistance'] > 0]  # Filter out zero values
  min_value = non_zero_values.min()  # Get minimum non-zero value
  min_row_index = non_zero_values[non_zero_values == min_value].index[0]  # Get the row index of the minimum value

  lower_bound = min_row_index-20000
  upper_bound = min_row_index+10000
  df2 = df.iloc[lower_bound:upper_bound, :]
  df2 = df2.drop(columns=['Milliseconds'])
  df2 = df2.reset_index(drop=True)
  df2['Milliseconds'] = (df2.index * step_interval_ms).astype(int)
  return df2

def plotter(df):
  plt.scatter(df["Milliseconds"].values,df["Roundabout.EgoPedEuclideanDistance"].values)
  plt.show()
  plt.scatter(df["Milliseconds"].values,df["VI_CarRealTime.Outputs.chassis_accelerations.longitudinal"].values)
  plt.show()
  plt.scatter(df["Milliseconds"].values,df["VI_CarRealTime.Outputs.chassis_velocities.longitudinal"].values)
  plt.show()

def agg_df(df_select_list):
  df_list = []
  for i in range(0, len(df_select_list)):
    df_list.append(loaded_dict[df_select_list[i]])

  fin_df = sum(df_list)/len(df_list)
  return fin_df

## 3. Combine csv files into single PKL file

In [ ]:
# sampling_rate = 100
# step_interval_ms = 1000 / sampling_rate
# #dir_name = "/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/Experiment Data (Unformatted)"
# dir_name = "/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/Experiment Data"
# dir_name_new= "/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/short_data"

# d = {}
# is_dir_name_new = os.path.isdir(dir_name_new)
# if is_dir_name_new==False:
#     os.mkdir(dir_name_new)

# #for participant in os.listdir(dir_name):
# participant = "Madhav"
# participant_path = os.path.join(dir_name, participant)
# print(participant_path)

# temp_path = os.path.join(dir_name_new, participant)
# print(temp_path)

# is_dir = os.path.isdir(temp_path)
# if is_dir==False:
#   os.mkdir(temp_path)

# # Ignore .DS_Store
# # if not os.path.isdir(participant_path):
# #     continue


# for csv_file in os.listdir(participant_path):
#     file_path = os.path.join(participant_path, csv_file)
#     if csv_file != ".DS_Store":
#        #print(csv_file)

#       df = pd.read_csv(file_path, encoding='ISO-8859-1', sep=';')
#       df2 = slim_data(df)

#       output_path = os.path.join(temp_path, csv_file)
#       # print("out_path")
#       # print(output_path)

#       a=csv_file.split(".")
#       d[a[0]] = df2

#       #print(d)
#       #df2.to_csv(output_path+".csv", index=False)

# # Save the dictionary to a file using pickle
# with open("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/store.pkl", "wb") as file:
#     pickle.dump(d, file)


##  4. Parse  Data


In [ ]:
# with open("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/store.pkl", "rb") as file:
#     loaded_dict = pickle.load(file)

# trial_df = pd.DataFrame.from_dict(trial_IVs)
# trial_df2 = trial_df.T
# trial_df2['combo'] = trial_df2.index
# print(trial_df2)

# col_list = ["Driver's Direction",   "Alert Combination", "Pedestrian Crossing"]
# dict_options = {}
# for i in range(0,len(col_list)):
#   a = trial_df2[col_list[i]].unique()
#   #print(a)
#   participant = "Madhav"
#   for j in range(0,len(a)):
#     f =trial_df2.loc[trial_df2[col_list[i]] == a[j]]
#     combos = f["combo"].tolist()
#     final_combo = [participant+"_"+str(s) for s in combos]
#     aggregated_df = agg_df(final_combo)
#     string_selection = a[j].replace(" ", "_")
#     dict_options["Madhav_"+string_selection] = aggregated_df

# # Save the dictionary to a file using pickle
# with open("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/options_store.pkl", "wb") as file:
#     pickle.dump(dict_options, file)

       Driver's Direction   Alert Combination Pedestrian Crossing  combo
1                    Left  Sign 2 + crosswalk      Straight ahead      1
2    Straight (left lane)           Crosswalk    To driver’s left      2
3   Straight (right lane)            No alert    To driver’s left      3
4   Straight (right lane)  Sign 2 + crosswalk   To driver’s right      4
5                    Left              Sign 3    To driver’s left      5
6    Straight (left lane)            No alert   To driver’s right      6
7                    Left  Sign 3 + crosswalk    To driver’s left      7
8                    Left  Sign 3 + crosswalk      Straight ahead      8
9   Straight (right lane)              Sign 3      Straight ahead      9
10                  Right           Crosswalk      Straight ahead     10
11                   Left              Sign 2   To driver’s right     11
12                  Right              Sign 3   To driver’s right     12
13                   Left           Crosswalk    To

## 5. Mount Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 6. Test Dashboard to verify inputs/outputs

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

with open("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/store.pkl", "rb") as file:
    loaded_dict = pickle.load(file)

key_list = []
for key, value in loaded_dict.items() :
    #print (key)
    key_list.append(key)

print(len(key_list))
cols = ['VI_CarRealTime.Outputs.chassis_accelerations.longitudinal','VI_CarRealTime.Outputs.chassis_velocities.longitudinal','Roundabout.EgoPedEuclideanDistance' ]

# Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Time Series Dashboard Test"),
    dcc.Dropdown(
        id='dataset-dropdown',
        options=[
            {'label': 'Madhav test 1', 'value': 'dataset1'},
            {'label': 'Madhav test 2 ', 'value': 'dataset2'},
            {'label': 'Madhav test 3', 'value': 'dataset3'},
        ],
        value='dataset1'
    ),
    html.Div(id='graphs-container')  # Container for the graphs
])

@app.callback(
    Output('graphs-container', 'children'),
    Input('dataset-dropdown', 'value')
)
def update_graphs(selected_dataset):
    # Select the appropriate dataset
    if selected_dataset == 'dataset1':
        df = loaded_dict["Madhav_1"]
    elif selected_dataset == 'dataset2':
        df = loaded_dict["Madhav_2"]
    elif selected_dataset == 'dataset3':
        df = loaded_dict["Madhav_3"]

    # Generate graphs for the selected dataset
    graphs = []
    for column in cols:
        fig = px.line(df, x='Milliseconds', y=column, title=f'{column} ({selected_dataset})')
        graphs.append(dcc.Graph(figure=fig))

    return graphs

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# test IPython widgets
# import ipywidgets as widgets
# from IPython.display import display
# import numpy as np

# random_numbers = [1,2,3]

# Dropdown_ = widgets.Dropdown(
#     options=random_numbers,
#     description='choose random number',
# )

# output = widgets.Output()

# def on_change(change):
#   if change['new'] > change['old']:
#     print('bigger')
#   else:
#     print('smaller')


# Dropdown_.observe(on_change, names='value')
# display(Dropdown_)

## 7. Creation of Final Dashboard

In [8]:
# Columns for visualization
cols = [
    'VI_CarRealTime.Outputs.chassis_accelerations.longitudinal',
    'VI_CarRealTime.Outputs.chassis_velocities.longitudinal',
    'Roundabout.EgoPedEuclideanDistance'
]

# Load the dictionary back from the file
with open("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/options_store.pkl", "rb") as file:
    dict_combos = pickle.load(file)

# for key, value in dict_combos.items() :
#     print (key)

  # Load the dictionary from the pickle file
with open("/content/drive/MyDrive/UWaterloo/CAPSTONE 🧢🪨/5. Simulator Study/Data/store.pkl", "rb") as file:
    loaded_dict = pickle.load(file)

df = loaded_dict["Madhav_6"]
Random_data = pd.DataFrame(columns= df.columns)


# Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Time Series Dashboard"),
    # Dropdown 1: Aggregation Type
    dcc.Dropdown(
        id='aggregation-type-dropdown',
        options=[
            {'label': 'Base Trials', 'value': 'nothing'},
            {'label': 'Drivers Direction', 'value': 'by_driver'},
            {'label': 'Alert Combination', 'value': 'by_alert'},
            {'label': 'Pedestrian Crossing', 'value': 'by_ped'}
        ],
        value='nothing',  # Default value
        placeholder="Select Aggregation Type",
        style={'width': '48%', 'display': 'inline-block'}
    ),
    # Dropdown 2: Options for each aggregation type
    dcc.Dropdown(
        id='aggregation-option-dropdown',
        placeholder="Select Specific Option",
        style={'width': '48%', 'display': 'inline-block'}
    ),
    # Graphs container
    html.Div(id='graphs-container')
])

# Callback to dynamically update options in the second dropdown based on aggregation type
@app.callback(
    Output('aggregation-option-dropdown', 'options'),
    Input('aggregation-type-dropdown', 'value')
)
def update_options(aggregation_type):
    if aggregation_type == 'nothing':
        return [{'label': f'Madhav Trial{i}', 'value': f'dataset{i}'} for i in range(1, 16)]
    elif aggregation_type == 'by_driver':
        return [
            {'label': 'Left', 'value': 'left'},
            {'label': 'Right', 'value': 'right'},
            {'label': 'Straight (left lane)', 'value': 'straight_left'},
            {'label': 'Straight (right lane)', 'value': 'straight_right'}
        ]

    elif aggregation_type == 'by_alert':
        return [
            {'label': 'Sign 2 + crosswalk', 'value': 's2_cw'},
            {'label': 'Crosswalk', 'value': 'cw'},
            {'label': 'No alert', 'value': 'na'},
            {'label': 'Sign 3', 'value': 's2'},
            {'label': 'Sign 3 + crosswalk', 'value': 's3_cw'},
            {'label': 'Sign 2', 'value': 's2'}
        ]

    elif aggregation_type == 'by_ped':
        return [
            {'label': 'Straight ahead', 'value': 'str_a'},
            {'label': 'Drivers Left', 'value': 'dl'},
            {'label': 'Drivers Right', 'value': 'dr'}
        ]
    return []

# Callback to generate graphs based on selected options
@app.callback(
    Output('graphs-container', 'children'),
    [
        Input('aggregation-type-dropdown', 'value'),
        Input('aggregation-option-dropdown', 'value')
    ]
)
def update_graphs(aggregation_type, aggregation_option):
    # Load the appropriate dataset based on aggregation type and option
    if aggregation_type == 'nothing':
      if aggregation_option is None:
        df = Random_data  # Output Random_data if aggregation_option is None
      elif aggregation_option.startswith('dataset'):
          dataset_number = aggregation_option.replace('dataset', '')  # Extract number from the option
          key = f"Madhav_{dataset_number}"
          df = loaded_dict.get(key)  # Safely get the dataset or fallback to Random_data

    elif aggregation_type == 'by_driver':

        if aggregation_option == 'left':
            df = dict_combos["Madhav_Left"]
        elif aggregation_option == 'right':
            df = dict_combos["Madhav_Right"]
        elif aggregation_option == 'straight_left':
            df = dict_combos["Madhav_Straight_(left_lane)"]
        elif aggregation_option == 'straight_right':
            df = dict_combos["Madhav_Straight_(right_lane)"]
        else:
            df = Random_data

    elif aggregation_type == 'by_alert':
        if aggregation_option == 's2_cw':
            df = dict_combos["Madhav_Sign_2_+_crosswalk"]
        elif aggregation_option == 'cw':
            df = dict_combos["Madhav_Crosswalk"]
        elif aggregation_option == 'na':
            df = dict_combos["Madhav_No_alert"]
        elif aggregation_option == 's3':
            df = dict_combos["Madhav_Sign_3"]
        elif aggregation_option == 's3_cw':
            df = dict_combos["Madhav_Sign_3_+_crosswalk"]
        elif aggregation_option == 's2':
            df = dict_combos["Madhav_Sign_2"]
        else:
            df = Random_data

    elif aggregation_type == 'by_ped':
        if aggregation_option == 'str_a':
            df = dict_combos["Madhav_Straight_ahead"]
        elif aggregation_option == 'dl':
            df = dict_combos["Madhav_To_driver’s_left"]
        elif aggregation_option == 'dr':
            df = dict_combos["Madhav_To_driver’s_right"]
        else:
            df = Random_data

    # Generate graphs for the selected dataset
    graphs = []
    for column in cols:
        if column in df.columns:  # Ensure the column exists
          unit = ""
          if column == "VI_CarRealTime.Outputs.chassis_accelerations.longitudinal":
            unit = "m/s^2"
          elif column == "VI_CarRealTime.Outputs.chassis_velocities.longitudinal":
            unit = "m/s"
          elif column == "Roundabout.EgoPedEuclideanDistance":
            unit = "m"
          avg =  str(round(df[column].mean(), 4))
          fig = px.line(df, x='Milliseconds', y=column, title=f'{column} => <b> {avg} {unit} avg </b>')
          graphs.append(dcc.Graph(figure=fig))

    return graphs

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>